In [2]:
import psycopg2
def connect_sql():  
    conn = psycopg2.connect(database="ssbms",
                            host="localhost",
                            user="postgres",
                            password="yourDBpass",
                            port="7643")
    # cursor = conn.cursor()
    return conn
def execute_sql(sql_code,conn):
    cursor = conn.cursor()
    try:
        cursor.execute(sql_code)    
        dat=[x for x in cursor]
        # if len(dat):
        #     dat="\n".join(dat)
        # else:
        #     dat="No data"
        
        return True,dat
    except:
        return False,""



In [8]:
conn=connect_sql()
relation_list={}
flag,data=execute_sql("select * from foreign_keys_view",conn)
relation=[]
for x in data:
    ss=f"{x[0]}.{x[1]} can be joined with {x[2]}.{x[3]}"
    if relation_list.get(x[0])==None:
        relation_list[x[0]]=[]
        relation_list[x[0]].append(ss)
    else:
        relation_list[x[0]].append(ss)
    if relation_list.get(x[2])==None:
        relation_list[x[2]]=[]
        relation_list[x[2]].append(ss)
    else:
        relation_list[x[2]].append(ss)
    relation.append(ss)

In [ ]:
conn=connect_sql()
x="alarms"
q1=f'''
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{x}' '''
flag,data=execute_sql(q1,conn)
data

In [ ]:
conn=connect_sql()
query='''SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public' AND table_type = 'BASE TABLE';
'''
flag,data=execute_sql(query,conn)

In [ ]:
table_name=[]
for x in data:
    xx=x[0]
    # print(xx)
    table_name.append(xx)
    # if xx[:4]=="emer":
    #     print(xx)
table_name="\n".join(table_name)
f=open("data/table_name.txt","w")
f.write(table_name)

In [ ]:
cnt=0
full_data=[]
for x in data:
    cnt+=1
    xx=x[0]
    q1=f'''SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{xx}' '''
    q2=f'''select * from {xx}'''
    _,data1=execute_sql(q1,conn)
    st=["Column name, Column data type"]
    for x2 in data1:
        st.append(f"{x2[0],x2[1]}")
    st="\n".join(st)

    _,data2=execute_sql(q2,conn)
    if len(data2)==0:
        data2=f"No data for {xx} table"
    else:
        st2=[]
        for x2 in data2:
            st2.append(str(x2))
        data2="\n".join(st2)
    ss=f"Table name: {xx}\nTable description:\n{st}\nTable data:\n{data2}"
    # print(ss)
  
    full_data.append(ss)




In [9]:
cnt=0
full_data=[]
for x in data:
    cnt+=1
    xx=x[0]
    q1=f'''SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{xx}' '''
    _,data1=execute_sql(q1,conn)
    st=[]
    for x2 in data1:
        st.append(f"{x2[0]}({x2[1]})")

    st=",".join(st)+"\n"+("\n".join(relation_list[xx]))

    
    ss=f"{xx}({st})"
  
    full_data.append(ss)




In [14]:
full_data[0]

'addresses(id(uuid),additional_number(integer),unit_number(integer),created_by(uuid),created_on(timestamp without time zone),building_number(integer),zip_code(integer),short_address(character varying),city_name(character varying),street_name(character varying),district_name(character varying)\naddresses.created_by can be joined with users.id\nfacilities.address_id can be joined with addresses.id)'

In [15]:
full_data2=full_data
open("/home/rtx3090/Desktop/shakib/sql/sqlcoder/data/tablename_with_col_relation.txt","w").write("\n".join(full_data2))

121999

In [ ]:
# f=open("reduced_data.txt","w")
# f.write(full_data)

In [ ]:
f=open("reduced_data.txt","r").read().split("\n")
f2=open(r"/home/rtx3090/Desktop/shakib/sql/sqlcoder/data/ssbms.sql","r").read().split("\n")